In [30]:
from finlab.crawler import table_date_range, update_table, to_pickle, out, commit,merge, date_range
import datetime
from io import StringIO
import requests
from bs4 import BeautifulSoup
import time

import numpy as np
#import matplotlib.pyplot as plt
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
#import pandas_datareader.data as web

import os
from openpyxl import load_workbook


def getStockData(parameter):
    url = "http://api.finmindtrade.com/api/v2/data"

    resp = requests.get(url, params=parameter)
    data = resp.json()
    data = pd.DataFrame(data["data"])
    return data

#get exchange rate list
url = "http://api.finmindtrade.com/api/v2/datalist?dataset=TaiwanExchangeRate"
data = requests.get(url)
data = data.json()
ExchangeRateList = pd.Series(data['data'])
#print(ExchangeRateList)

In [31]:
#the data is small, just grab from scratch to today, no need to append into pickle

combined_df = pd.DataFrame()

for i in range ( 0,len(ExchangeRateList)):
    parameter = {
    "dataset": "TaiwanExchangeRate",
    "data_id": ExchangeRateList[i],
    "date": "2000-01-01",
    "end_date": "2020-06-12"
    }

    df_tmp=getStockData(parameter)
    combined_df = combined_df.append(df_tmp)
    print(str(i) + "/" +str(len(ExchangeRateList)),len(combined_df))
    time.sleep(3)



0/19 3586
1/19 7172
2/19 10758
3/19 14344
4/19 17930
5/19 21516
6/19 25102
7/19 28688
8/19 32274
9/19 35860
10/19 39446
11/19 43032
12/19 46618
13/19 50204
14/19 53790
15/19 57376
16/19 60962
17/19 64548
18/19 64656


In [32]:
combined_df=combined_df.set_index('date')
combined_df.index = pd.to_datetime(combined_df.index)
combined_df=combined_df.reset_index()
combined_df=combined_df.set_index(['currency', 'date'])
combined_df.shape

(64656, 4)

In [33]:
combined_df

cash_buy  cash_sell  spot_buy  spot_sell
currency date                                                
AUD      2006-01-02     23.74      24.49   -99.000    -99.000
         2006-01-03     23.70      24.44    23.960     24.160
         2006-01-04     23.63      24.37    23.890     24.090
         2006-01-05     23.57      24.31    23.830     24.030
         2006-01-06     23.65      24.39    23.910     24.110
         2006-01-09     23.68      24.42    23.940     24.140
         2006-01-10     23.62      24.36    23.880     24.080
         2006-01-11     23.70      24.44    23.960     24.160
         2006-01-12     23.70      24.44    23.960     24.160
         2006-01-13     23.69      24.43    23.950     24.150
         2006-01-16     23.74      24.49    24.000     24.200
         2006-01-17     23.79      24.54    24.050     24.250
         2006-01-18     23.71      24.46    23.970     24.170
         2006-01-19     23.61      24.35    23.870     24.070
         2006-01-20     23.71      24.46    23.970     24.170
         2006-01-23     23.72      24.47    23.980     24.180
         2006-01-24     23.66      24.40    23.920     24.120
         2006-01-25     23.67      24.41    23.930     24.130
         2006-01-26     23.79      24.54    24.050     24.250
         2006-01-27     23.70      24.44    23.960     24.160
         2006-02-03     23.82      24.57    24.080     24.280
         2006-02-06     23.60      24.34    23.870     24.070
         2006-02-07     23.60      24.34    23.860     24.060
         2006-02-08     23.53      24.27    23.790     23.990
         2006-02-09     23.66      24.40    23.920     24.120
         2006-02-10     23.55      24.29    23.810     24.010
         2006-02-13     23.50      24.24    23.760     23.960
         2006-02-14     23.46      24.20    23.720     23.920
         2006-02-15     23.63      24.37    23.890     24.090
         2006-02-16     23.58      24.32    23.840     24.040
...                       ...        ...       ...        ...
ZAR      2020-05-04     -1.00      -1.00     1.551      1.631
         2020-05-05     -1.00      -1.00     1.581      1.661
         2020-05-06     -1.00      -1.00     1.583      1.663
         2020-05-07     -1.00      -1.00     1.566      1.646
         2020-05-08     -1.00      -1.00     1.578      1.658
         2020-05-11     -1.00      -1.00     1.594      1.674
         2020-05-12     -1.00      -1.00     1.579      1.659
         2020-05-13     -1.00      -1.00     1.576      1.656
         2020-05-14     -1.00      -1.00     1.571      1.651
         2020-05-15     -1.00      -1.00     1.587      1.667
         2020-05-18     -1.00      -1.00     1.582      1.662
         2020-05-19     -1.00      -1.00     1.596      1.676
         2020-05-20     -1.00      -1.00     1.599      1.679
         2020-05-21     -1.00      -1.00     1.631      1.711
         2020-05-22     -1.00      -1.00     1.641      1.721
         2020-05-25     -1.00      -1.00     1.663      1.743
         2020-05-26     -1.00      -1.00     1.678      1.758
         2020-05-27     -1.00      -1.00     1.676      1.756
         2020-05-28     -1.00      -1.00     1.685      1.765
         2020-05-29     -1.00      -1.00     1.669      1.749
         2020-06-01     -1.00      -1.00     1.679      1.759
         2020-06-02     -1.00      -1.00     1.679      1.759
         2020-06-03     -1.00      -1.00     1.714      1.794
         2020-06-04     -1.00      -1.00     1.715      1.795
         2020-06-05     -1.00      -1.00     1.739      1.819
         2020-06-08     -1.00      -1.00     1.733      1.813
         2020-06-09     -1.00      -1.00     1.730      1.810
         2020-06-10     -1.00      -1.00     1.746      1.826
         2020-06-11     -1.00      -1.00     1.743      1.823
         2020-06-12     -1.00      -1.00     1.701      1.781

[64656 rows x 4 columns]

In [34]:
combined_df.to_pickle("./history/tables/Exchange_Rate.pkl")
